## for local conda supported

In [ ]:
from optimum.onnxruntime import ORTModelForCausalLM

model = ORTModelForCausalLM.from_pretrained(
    "Neupane9Sujal/GPT2-chatbot-nepali-for-disaster",
    export=True  # Converts to ONNX format
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/94.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/cache_utils.py:556: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  or not self.key_cache[layer_idx].numel()  # the layer has no cache
/usr/local/lib/python3.11/dist-packages/transformers/models/gpt2/modeling_gpt2.py:1096: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if sequence_length != 1:
/usr/local/lib/python3.11/dist-packages/transformers/cache_utils.py:539: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Neupane9Sujal/GPT2-chatbot-nepali-for-disaster")

In [ ]:
%%time
prompt = "काठमाडौंमा भूकम्प आउँदा के गर्नुपर्छ?"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate text
outputs = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=300,  # Maximum length of generated text
    num_return_sequences=1,  # Number of sequences to generate
    temperature=0.7,  # Controls randomness (lower = more deterministic)
    do_sample=True  # Enable sampling
)

# Decode and print the output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CPU times: user 15.5 s, sys: 12.7 ms, total: 15.5 s
Wall time: 18.5 s


In [ ]:
print(generated_text.split('<bot>:')[1])

 काठमाडौंका मानिसहरूले सुधारिनु पर्दछ, आपतकालीन किट राख्नुहोस्, भूकम्पको क्रममा स्थानीय सतर्कताहरूको पालना गर्नुहोस्। ्गर्नु�


In [ ]:
## Import necessary libraries
import torch
from torch.utils.data import Dataset, DataLoader
import json
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.optim import AdamW
from transformers import DataCollatorForSeq2Seq

## Tokenize the dataset
class TokenizedDataset(Dataset):
    def __init__(self, path, tokenizer):
        self.data = json.load(open(path, "r"))
        self.tokenizer = tokenizer

        self.conversations = []
        for intent in self.data:
            question = intent['question']
            answer = intent['answer']

            # response = responses[i % len(responses)]  # Handle cases where there are more patterns than responses
            conversation = "<startofstring> " + question + " <bot>: " + answer + " <endofstring>"
            self.conversations.append(conversation)

        self.x_encoded = tokenizer.batch_encode_plus(
            self.conversations,
            max_length=512,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        self.input_ids = self.x_encoded['input_ids']
        self.attention_mask = self.x_encoded['attention_mask']

    def __len__(self):
        return len(self.conversations)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx]
        }

device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = GPT2Tokenizer.from_pretrained("Neupane9Sujal/GPT2-chatbot-nepali-for-disaster")
tokenizer.add_special_tokens({"pad_token":"<pad>",
                            "bos_token":"<startofstring>",
                            "eos_token":"<endofstring>"})
tokenizer.add_tokens(['<bot>:'])

## Initialize the model
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))
model.to(device)

optimizer = AdamW(model.parameters(), lr = 1e-4)

dataset = TokenizedDataset(path='/content/disaster_chatbot_data_nepali.json', tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/94.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [ ]:
tokenizer.save_pretrained('/content/model')

('/content/model/tokenizer_config.json',
 '/content/model/special_tokens_map.json',
 '/content/model/vocab.json',
 '/content/model/merges.txt',
 '/content/model/added_tokens.json')

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)
dataloader = DataLoader(dataset, batch_size=8,collate_fn=data_collator, shuffle=True)

In [ ]:
from torch.optim import AdamW
# from torch.amp import autocast, GradScaler
import tqdm

# scaler = GradScaler()

def infer(inp):
  model.eval()
  inp = "<startofstring> "+inp+" <bot>: "
  inp = tokenizer(inp,return_tensors='pt')
  X = inp['input_ids'].to(device)
  a = inp['attention_mask'].to(device)
  output = model.generate(X, attention_mask=a, max_length=512)
  output = tokenizer.decode(output[0],skip_special_tokens=True)
  return output


def train(dataset, model, optim):
  num_epochs = 20
  model.to(device)
  model.train()
  for i in tqdm.tqdm(range(num_epochs)):
    for batch in dataset:
      X = batch['input_ids'].to(device)
      a = batch['attention_mask'].to(device)
      optim.zero_grad()
      # with autocast(): ## for mixed precision training
      out = model(X, attention_mask=a,labels=X)
      loss=out.loss
      # scaler.scale(loss).backward()
      loss.backward()
      # scaler.step(optim)
      optim.step()
      # scaler.update()
  model.save_pretrained('/content/model')



train(dataloader,model, optimizer)

100%|██████████| 20/20 [39:19<00:00, 117.96s/it]


In [ ]:
# भूकम्प आउँदा के गर्नुपर्छ?
print("infer from model : ")
while True:
  inp = input()
  print(infer(inp))

infer from model : 
के म बाढी पछि नल पानी प्रयोग गर्न सक्छु?


NameError: name 'infer' is not defined

In [ ]:
from huggingface_hub import notebook_login, HfApi

notebook_login()

In [ ]:
api = HfApi()

api.upload_folder(
    folder_path='/content/model',
    path_in_repo=".",
    repo_id="Neupane9Sujal/GPT2-chatbot-nepali-for-disaster",
    repo_type='model',
    create_pr=1
)

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Neupane9Sujal/GPT2-chatbot-nepali-for-disaster/commit/4f4442de37418616de842359a55466188c091cca', commit_message='Upload folder using huggingface_hub', commit_description='', oid='4f4442de37418616de842359a55466188c091cca', pr_url='https://huggingface.co/Neupane9Sujal/GPT2-chatbot-nepali-for-disaster/discussions/2', repo_url=RepoUrl('https://huggingface.co/Neupane9Sujal/GPT2-chatbot-nepali-for-disaster', endpoint='https://huggingface.co', repo_type='model', repo_id='Neupane9Sujal/GPT2-chatbot-nepali-for-disaster'), pr_revision='refs/pr/2', pr_num=2)

In [ ]:
!pip install optimum[onnxruntime]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 594.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 